## 5.1.2 Phép toán Max-pooling

In [1]:
from IPython.display import HTML
HTML('<img src="https://d3ansictanv2wj.cloudfront.net/Figure_4-87c227113cdd0b73d842267404d4aa00.gif"/>')

Trong ví dụ về convnet, bạn để ý thấy size của feature map sẽ **giảm 1 nửa sau mỗi MaxPooling2D layer**. 
```
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
```

Từ (26,26), kích thước của feature map sau max_pooling2d còn (13,13) 

Max pooling cũng lấy các cụm pixel (sliding windows) từ input feature maps và **output giá trị max** của mỗi kênh. Nó hoạt động khá giống với convolution, nhưng thay vì sử dụng kernel cho phép transform, max pool **dùng 1 phép toán cố định : max**. 1 điểm khác biệt lớn nữa so với convolution, là maxpool thường lấy các window 2x2 và stride 2, giúp giảm độ lớn các feature map xuống 2 lần. Trong khi đó, convolution thường dùng 3x3 window và stride = 1

Tại sao phải sử dụng max pooling layer ? Giảm feature size có tác dụng gì? Ta sẽ thử tạo network như vậy :

In [3]:
from keras import models, layers

In [4]:
model_no_max_pool = models.Sequential()
model_no_max_pool.add(layers.Conv2D(32,(3,3),activation='relu', input_shape=(28,28,1)))
model_no_max_pool.add(layers.Conv2D(64,(3,3),activation='relu'))
model_no_max_pool.add(layers.Conv2D(64,(3,3),activation='relu'))

model_no_max_pool.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Có 2 điểm không ổn trong network này :
- 28x28 --> 22x22 : **các windows 3x3 của layer thứ 3 chỉ chứa các thông tin từ các windows 7x7 của ảnh ban đầu**. Các thuộc tính bao quát của convnet sẽ vẫn rất nhỏ so với input, và có thể sẽ không đủ cho bài toán classify ( cố gắng nhận ra chữ số trên ảnh 28x28 bằng cách nhìn qua các windows 7x7). Các thuộc tính từ convolution layer cuối cần chứa toàn thể thuộc tính của input
- Feature map cuối có tất cả 22 * 22 * 64 = 31000 hệ số cho mỗi 1 sample --> Rất lớn. Nếu chúng ta Flatten và đặt 1 Dense layer với size 512 liền sau đó, Dense layer đó sẽ có **15.8 triệu parameters** --> **Quá lớn cho 1 model nhỏ --> Overfitting**

Tóm lại, lý do chúng ta **downsampling** đơn giản là **giảm số lương các trọng số** của feature map ( tính toán nhanh hơn và giảm over-fit), cũng như phân tầng các filters bằng cách làm cho các convolution layer theo sau **bao quát** được thuộc tính từ các **windows lớn hơn**

Lưu ý rằng max-pool **không phải là cách duy nhất** để downsampling. Ta cũng có thể sử dụng **strides** trong convolution layer, hoặc **average pooling** (thay vì lấy giá trị max trong cụm pixel, average-pooling lấy **giá trị trung bình** của cụm đó). Tuy vậy, **max-pool thường sẽ cho kết quả tốt hơn**. Hiểu 1 cách đơn giản, max pool sẽ **giữ lại thuộc tính nổi trội nhất** trong cụm, trong khi giá trị trung bình (**average pooling**) hay những cụm pixel tách biệt (**stride**) **có thể sẽ làm gỉam / mất 1 số thuộc tính** của object 

Bạn đã hiểu được những điều cơ bản về convnets : feature maps, convolution, max pool và cách tạo 1 convnet cho bài toán MNIST. Bây giờ chúng ta sẽ đi sâu vào thực hành với ví dụ thực tế hơn 

Link tham khảo :
- Kernel - Hiểu hơn về kernel [Hit Or Miss Opencv](https://docs.opencv.org/master/db/d06/tutorial_hitOrMiss.html)
- Convolution neural network - Udacity [youtube](https://youtu.be/ISHGyvsT0QY) - Vi deo rất trực quan
- Convolution neural network Intel neon (khá hay) [youtube](https://www.youtube.com/watch?v=xg2ajb3csgk)
- Cs231n - Rất chi tiết [link](http://cs231n.github.io/convolutional-networks/#conv)